In [1]:
from azureml.core import Workspace, Experiment
ws=Workspace.from_config()
#ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-130809
Azure region: southcentralus
Subscription id: dafd62fe-51d3-4671-af50-6bac2f001b5d
Resource group: aml-quickstarts-130809


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpucluster"
# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
from azureml.train.hyperdrive import normal, uniform, choice
ps = RandomParameterSampling( {
        "--C": uniform(0.05, 0.1),
        "--max_iter": choice(16, 32, 64, 128)})### YOUR CODE HERE ###

# Specify a Policy
policy =  BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5)   ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', 
                    entry_script='train.py', 
                    compute_target=cpu_cluster)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (estimator = est,
                             hyperparameter_sampling = ps,
                             policy = policy,
                             primary_metric_name = "Accuracy",
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 50,
                             max_concurrent_runs = 3
                             )### YOUR CODE HERE ###

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
experiment = Experiment(ws,'udacity-project')
hyperdrive_run = experiment.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [11]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print('best run ID: ', best_run.id)
print('\n Accuracy: ',best_run_metrics['Accuracy'])
#best_run
#joblib.dump(value = best_run, filename = 'bestrun.pkl')
model = best_run.register_model(model_name='bankmodel', model_path='outputs/model.joblib')
#model = best_run.register_model(model_name='hd-model',model_path='outputs/mybestrun.joblib', tags={'Method':'Hyperdrive'},properties={'Accuracy': best_run_metrics['Accuracy']})

best run ID:  HD_fd412d61-fe79-4b3c-a4cf-27650d9a536d_6

 Accuracy:  0.9111785533636824


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
URL = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=URL)

In [7]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds) ### YOUR DATA OBJECT HERE ###)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)
dataset = pd.concat([x_train,y_train],axis=1)

In [8]:
dataset.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
20086,38,1,0,1,0,6,2,258,1,999,...,1,0,1,0,0,0,0,0,0,0
17643,43,1,0,0,0,5,5,38,2,999,...,1,0,0,0,0,0,0,0,1,0
8998,32,1,0,1,0,8,2,66,7,999,...,1,0,0,0,0,0,0,1,0,0
8801,27,0,0,0,0,4,5,48,2,999,...,0,0,0,1,0,0,0,0,0,0
9080,36,1,0,1,0,7,3,191,2,999,...,0,0,0,0,0,0,0,1,0,0


In [9]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5)

In [10]:
# Submit your automl run

### YOUR CODE HERE ###
#import azureml.core
#from azureml.core.experiment import Experiment
#experiment = Experiment(ws,'udacity-project')
run = experiment.submit(config = automl_config, show_output=False)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [14]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [17]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
import joblib
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, "fitted_automl_model.joblib")

Run(Experiment: udacity-project,
Id: AutoML_7a61ff59-97dc-40c6-b7b3-e2fd0aeb0d47_51,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  l1_ratio=0.836734693877551,
                                                                                                  learning_rate='constant',
                          

['fitted_automl_model.joblib']

In [3]:
cpu_cluster.delete()